# Personal Injury Settlement Analysis - SPY (S&P 500) - Stacked Simulation

Analyzing retirement options for a $677,530 personal injury settlement using efficient stacked simulations with SPY (S&P 500) instead of VT.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from finsim.stacked_simulation import (
    create_scenario_config,
    simulate_stacked_scenarios,
    analyze_confidence_thresholds,
    summarize_confidence_thresholds
)

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

print("Loaded finsim stacked simulation functionality")

## Settlement Details

- **Total Settlement**: $677,530
- **Annuity Cost**: $527,530
- **Immediate Cash**: $150,000

### Annuity Options
1. **Annuity A**: Life with 15-year guarantee - $3,516.29/month ($42,195/year)
2. **Annuity B**: 15 years guaranteed - $4,057.78/month ($48,693/year)
3. **Annuity C**: 10 years guaranteed - $5,397.12/month ($64,765/year)

**Important**: Personal injury annuities are tax-free.

In [ ]:
# Settlement parameters
TOTAL_SETTLEMENT = 677_530
ANNUITY_COST = 527_530
IMMEDIATE_CASH = TOTAL_SETTLEMENT - ANNUITY_COST

# Annuity annual payments
ANNUITY_A_ANNUAL = 3_516.29 * 12  # $42,195
ANNUITY_B_ANNUAL = 4_057.78 * 12  # $48,693
ANNUITY_C_ANNUAL = 5_397.12 * 12  # $64,765

print(f"Total Settlement: ${TOTAL_SETTLEMENT:,}")
print(f"Annuity Cost: ${ANNUITY_COST:,}")
print(f"Immediate Cash: ${IMMEDIATE_CASH:,}")
print(f"\nAnnuity A: ${ANNUITY_A_ANNUAL:,.0f}/year (life with 15-yr guarantee)")
print(f"Annuity B: ${ANNUITY_B_ANNUAL:,.0f}/year (15 years)")
print(f"Annuity C: ${ANNUITY_C_ANNUAL:,.0f}/year (10 years)")

## Define Scenarios with SPY

In [ ]:
# Create scenario configurations
scenarios = [
    create_scenario_config(
        name="100% Stocks (SPY)",
        initial_portfolio=TOTAL_SETTLEMENT,
        has_annuity=False
    ),
    create_scenario_config(
        name="Annuity A + SPY",
        initial_portfolio=IMMEDIATE_CASH,
        has_annuity=True,
        annuity_type="Life Contingent with Guarantee",
        annuity_annual=ANNUITY_A_ANNUAL,
        annuity_guarantee_years=15
    ),
    create_scenario_config(
        name="Annuity B + SPY",
        initial_portfolio=IMMEDIATE_CASH,
        has_annuity=True,
        annuity_type="Fixed Period",
        annuity_annual=ANNUITY_B_ANNUAL,
        annuity_guarantee_years=15
    ),
    create_scenario_config(
        name="Annuity C + SPY",
        initial_portfolio=IMMEDIATE_CASH,
        has_annuity=True,
        annuity_type="Fixed Period",
        annuity_annual=ANNUITY_C_ANNUAL,
        annuity_guarantee_years=10
    )
]

# Display scenario details
for scenario in scenarios:
    print(f"\n{scenario['name']}:")
    print(f"  Initial portfolio: ${scenario['initial_portfolio']:,}")
    if scenario['has_annuity']:
        print(f"  Annuity income: ${scenario['annuity_annual']:,.0f}/year")
        print(f"  Guarantee period: {scenario['annuity_guarantee_years']} years")

## Base Parameters with SPY Characteristics

SPY (S&P 500) historically has:
- Slightly higher expected returns than total world stock market
- Similar or slightly lower volatility due to large-cap US focus
- Higher dividend yield than VT

In [ ]:
# Base parameters for all scenarios - adjusted for SPY
base_params = {
    "current_age": 65,
    "gender": "Male",
    "social_security": 24_000,
    "pension": 0,
    "employment_income": 0,
    "retirement_age": 65,
    "expected_return": 7.5,  # 7.5% expected return for SPY (slightly higher than VT)
    "return_volatility": 17.0,  # 17% volatility for SPY (slightly lower than VT)
    "dividend_yield": 2.0,  # 2.0% dividend yield for SPY (slightly higher than VT)
    "state": "CA",
    "include_mortality": True,
}

# Display parameters
print("Base Parameters (SPY):")
print(f"  Age: {base_params['current_age']}")
print(f"  Gender: {base_params['gender']}")
print(f"  Social Security: ${base_params['social_security']:,}/year")
print(f"  Expected Return: {base_params['expected_return']}%")
print(f"  Volatility: {base_params['return_volatility']}%")
print(f"  Dividend Yield: {base_params['dividend_yield']}%")
print(f"  State: {base_params['state']}")

## Run Stacked Simulations

Using the efficient stacked approach - all scenarios run together with shared tax calculations.

In [ ]:
# Define spending levels to test
spending_levels = list(range(30_000, 105_000, 5_000))

print(f"Testing {len(spending_levels)} spending levels: ${min(spending_levels):,} to ${max(spending_levels):,}")
print(f"Running {len(scenarios)} scenarios stacked together")
print(f"Simulations per scenario: 2,000")
print(f"\nThis uses only {len(spending_levels) * 30} tax calculations instead of {len(scenarios) * 2000 * 30 * len(spending_levels):,}")
print(f"Speedup: {(len(scenarios) * 2000):,}x")
print("\nRunning simulations...")

In [ ]:
%%time
# Run the stacked simulations
results = simulate_stacked_scenarios(
    scenarios=scenarios,
    spending_levels=spending_levels,
    n_simulations=2000,
    n_years=30,
    base_params=base_params,
    include_percentiles=True,
    random_seed=42
)

print(f"\nCompleted {len(results)} scenario-spending combinations")

## Analyze Results

In [ ]:
# Convert results to DataFrame
df = pd.DataFrame(results)

# Display sample results
print("Sample results:")
display(df.head(10))

## Find Sustainable Spending at Various Confidence Levels

In [ ]:
# Analyze confidence thresholds
confidence_levels = [90, 75, 50, 25, 10]
scenario_names = [s["name"] for s in scenarios]

# Create summary table
summary_df = summarize_confidence_thresholds(
    results=results,
    scenarios=scenario_names,
    confidence_levels=confidence_levels
)

# Format the table for display
formatted_summary = summary_df.copy()
for col in formatted_summary.columns[1:]:
    formatted_summary[col] = formatted_summary[col].apply(lambda x: f"${x:,.0f}")

print("\nSUSTAINABLE SPENDING AT VARIOUS CONFIDENCE LEVELS (SPY)")
print("=" * 70)
print("(All amounts in 2025 dollars)\n")
display(formatted_summary)

## Visualize Success Rates

In [ ]:
# Create visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Left plot: All scenarios on one chart
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
for idx, scenario_name in enumerate(scenario_names):
    scenario_df = df[df["scenario"] == scenario_name]
    ax1.plot(scenario_df["spending"] / 1000, 
             scenario_df["success_rate"] * 100,
             color=colors[idx], linewidth=2.5, 
             marker='o', markersize=5,
             label=scenario_name, alpha=0.8)

# Add confidence level lines
for confidence in [90, 75, 50, 25]:
    ax1.axhline(y=confidence, color='gray', linestyle='--', alpha=0.3)
    ax1.text(102, confidence, f'{confidence}%', fontsize=9, va='center')

ax1.set_xlabel('Annual Spending ($1000s)', fontsize=12)
ax1.set_ylabel('Success Rate (%)', fontsize=12)
ax1.set_title('Success Rate vs Annual Spending (SPY)', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.set_ylim(0, 105)
ax1.set_xlim(28, 102)
ax1.legend(loc='upper right', fontsize=10)

# Right plot: 90% confidence spending comparison
confidence_90 = []
for scenario_name in scenario_names:
    thresholds = analyze_confidence_thresholds(results, scenario_name, [90])
    confidence_90.append(thresholds[90])

bars = ax2.bar(range(len(scenario_names)), confidence_90, color=colors, alpha=0.7)
ax2.set_xticks(range(len(scenario_names)))
ax2.set_xticklabels([s.replace(' + SPY', '\n+ SPY') for s in scenario_names], 
                     rotation=0, ha='center')
ax2.set_ylabel('Sustainable Spending ($)', fontsize=12)
ax2.set_title('90% Confidence Sustainable Spending (SPY)', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar, value in zip(bars, confidence_90):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 1000,
             f'${value:,.0f}', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

## Detailed Analysis at Key Confidence Levels

In [ ]:
# Analyze key confidence levels
print("\nKEY FINDINGS (SPY)")
print("=" * 70)

for confidence in [90, 75, 50]:
    print(f"\nAt {confidence}% confidence level:")
    best_spending = 0
    best_scenario = ""
    
    for scenario_name in scenario_names:
        thresholds = analyze_confidence_thresholds(results, scenario_name, [confidence])
        spending = thresholds[confidence]
        print(f"  {scenario_name}: ${spending:,.0f}/year")
        
        if spending > best_spending:
            best_spending = spending
            best_scenario = scenario_name
    
    print(f"  → Best option: {best_scenario}")

## Portfolio Percentiles Analysis

In [ ]:
# Analyze portfolio outcomes at 90% confidence spending levels
print("\nPORTFOLIO OUTCOMES AT 90% CONFIDENCE SPENDING (SPY)")
print("=" * 70)

for scenario_name in scenario_names:
    # Find 90% confidence spending
    thresholds = analyze_confidence_thresholds(results, scenario_name, [90])
    target_spending = thresholds[90]
    
    # Find closest result
    scenario_results = df[df["scenario"] == scenario_name]
    closest_idx = (scenario_results["spending"] - target_spending).abs().idxmin()
    result = scenario_results.loc[closest_idx]
    
    print(f"\n{scenario_name} at ${target_spending:,.0f}/year:")
    print(f"  Success rate: {result['success_rate']:.1%}")
    print(f"  Median final portfolio: ${result['median_final']:,.0f}")
    print(f"  10th percentile: ${result['p10_final']:,.0f}")
    print(f"  90th percentile: ${result['p90_final']:,.0f}")

## Risk-Return Tradeoff

In [ ]:
# Create risk-return plot
fig, ax = plt.subplots(figsize=(10, 6))

# For each scenario, plot confidence curve
confidence_range = [95, 90, 85, 80, 75, 70, 65, 60, 55, 50, 45, 40, 35, 30, 25, 20, 15, 10]

for idx, scenario_name in enumerate(scenario_names):
    thresholds = analyze_confidence_thresholds(results, scenario_name, confidence_range)
    spending_at_confidence = [thresholds[c] for c in confidence_range]
    
    ax.plot(confidence_range, np.array(spending_at_confidence) / 1000,
            color=colors[idx], linewidth=2.5,
            marker='o', markersize=4,
            label=scenario_name, alpha=0.8)

ax.set_xlabel('Confidence Level (%)', fontsize=12)
ax.set_ylabel('Sustainable Spending ($1000s/year)', fontsize=12)
ax.set_title('Risk-Return Tradeoff: Confidence vs Sustainable Spending (SPY)', 
             fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend(loc='upper right', fontsize=10)
ax.invert_xaxis()  # Higher confidence on the left

# Add vertical lines at key confidence levels
for conf in [90, 75, 50]:
    ax.axvline(x=conf, color='gray', linestyle='--', alpha=0.3)

plt.tight_layout()
plt.show()

## Comparison: SPY vs VT Performance

With SPY's characteristics:
- **Higher expected return (7.5% vs 7.0%)**: Should lead to better long-term performance
- **Lower volatility (17% vs 18%)**: Should provide more consistent outcomes
- **Higher dividend yield (2.0% vs 1.8%)**: Slightly more income generation

## Summary and Recommendations

In [ ]:
print("\nSUMMARY AND RECOMMENDATIONS (SPY)")
print("=" * 70)

print("\n1. CONSERVATIVE APPROACH (90% confidence):")
print("   → Annuity A (Life with 15-yr guarantee) likely provides highest sustainable spending")
print("   → SPY's higher expected return improves stock-only option")
print("   → Provides lifetime income protection")

print("\n2. MODERATE APPROACH (75% confidence):")
print("   → Annuity A still likely optimal")
print("   → 100% SPY performs better than VT due to higher returns")

print("\n3. BALANCED APPROACH (50% confidence):")
print("   → 100% SPY may become optimal")
print("   → Higher potential with less volatility than VT")

print("\n4. SPY-SPECIFIC CONSIDERATIONS:")
print("   • US-only exposure (no international diversification)")
print("   • Large-cap focus (S&P 500 companies)")
print("   • Historically strong performance but past != future")
print("   • Lower expense ratio than VT (0.09% vs 0.07%)")

print("\n5. KEY CONSIDERATIONS (SAME AS VT):")
print("   • Personal injury annuities are TAX-FREE (major advantage)")
print("   • Annuity A provides lifetime protection against longevity risk")
print("   • 100% Stocks offers more flexibility and potential upside")
print("   • Health status and life expectancy are critical factors")
print("   • Consider partial strategies (e.g., 50% annuity, 50% stocks)")

print("\n" + "=" * 70)

## Save Results

In [ ]:
# Save detailed results
df.to_csv('settlement_analysis_spy_results.csv', index=False)
print("Results saved to settlement_analysis_spy_results.csv")

# Save summary table
summary_df.to_csv('settlement_spy_confidence_summary.csv', index=False)
print("Summary saved to settlement_spy_confidence_summary.csv")